Packages:

In [5]:
import glob
import os
import warnings
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping


 

In [6]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [7]:
# Specify the file names and DataFrame variable names
file_names = ["train_df_2.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Load train_df as a DataFrame
train_df = dataframes[0]
print("DataFrames have been loaded successfully.")

DataFrames have been loaded successfully.


In [8]:
train_df['severity'].value_counts()


severity
0.0    7000
1.0    7000
Name: count, dtype: int64

In [9]:
train_df.dtypes

study_id           int64
severity         float64
condition          int64
level             object
series_id        float64
x                float64
y                float64
image_path        object
missing_image       bool
dtype: object

In [10]:
# # end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=20, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))



Using the whole data set


In [11]:
train_df


,study_id,severity,condition,level,series_id,x,y,image_path,missing_image
0,4003253,0.0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/1054713880/3.dcm,False
1,4003253,0.0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/702807833/3.dcm,False
2,4003253,0.0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/1054713880/14...,False
3,4646740,1.0,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/2.dcm,False
4,4646740,1.0,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/1.dcm,False
...,...,...,...,...,...,...,...,...,...
13995,4290709089,0.0,0,l5/s1,4.237840e+09,211.813953,259.534884,data/train_images_origin/4290709089/3274612423...,False
13996,4290709089,0.0,0,l5/s1,4.237840e+09,211.813953,259.534884,data/train_images_origin/4290709089/3274612423...,False
13997,4290709089,0.0,0,l5/s1,4.237840e+09,211.813953,259.534884,data/train_images_origin/4290709089/4237840455...,False
13998,4290709089,0.0,0,l5/s1,4.237840e+09,211.813953,259.534884,data/train_images_origin/4290709089/4237840455...,False


Ttrasformaiton and dataloader (do not change over models)

In [12]:
# define transformation and class
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
# %pip install torch torchvision
import torchvision.models as models
from torchvision import transforms
import pydicom
# %pip install opencv-python
import cv2
import pandas as pd

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),  # Uncomment if you want to use horizontal flipping
    # transforms.RandomRotation(10),     # Uncomment if you want to use rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading MRI images
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        # Ensure severity is in integer format
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']  # Use severity for the label

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')  # Normalize

        # Convert the image to RGB if it is grayscale
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Apply transformations including augmentation
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()  # Return label as tensor

    def __len__(self):
        return len(self.data)


In [13]:
train_data = train_df

training model resnet50 sotp if validation and train go apart

* Setting the model

In [14]:
import mlflow
import mlflow.pytorch
import random
# Set random seed for reproducibility
seed = 42  # You can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)  # Set seed for numpy
random.seed(seed)  # Set seed for random module

# Create a new experiment
experiment_name = "Resnet50_2_cat_basic"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///c:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/721936859901599929', creation_time=1730966620396, experiment_id='721936859901599929', last_update_time=1730966620396, lifecycle_stage='active', name='Resnet50_2_cat_basic', tags={}>

* some settings: train validation,  Early stopping parameters

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
# %pip install keras
# %pip install tensorflow
# from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping
import numpy as np  # Import numpy for setting the random seed

# Create the dataset
dataset = MRIDataset(data=train_data, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 20
# Early stopping parameters

# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []

# Early stopping parameters
stop_threshold = 0.2  # Threshold for validation loss to diverge from training loss
diverge_count = 0
max_diverge_count = 3  # Number of epochs validation loss is allowed to diverge

# Calculate number of layers in the model
num_layers = len(list(model.children()))


* setting mlflow: end running new model

In [ ]:
# Start MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("learning_rate", 0.0001)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 4)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("model_architecture", "ResNet-50")
    mlflow.log_param("input_size", "224x224")
    mlflow.log_param("num_layers", num_layers)
    mlflow.log_param("activation_function", "ReLU")
    mlflow.log_param("pretrained_weights", "IMAGENET1K_V1")

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_cel_train = 0.0
        running_loss_mse_train = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(images)
            
            # Cross Entropy Loss
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()
            
            # Backward pass and optimization
            loss_cel.backward()
            optimizer.step()

        # Calculate average losses
        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)

        # Log training losses to MLflow
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)


        # Validation phase
        model.eval()
        running_loss_cel_val = 0.0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                # Cross Entropy Loss for validation
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()
                
        # Calculate validation losses
        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)

        
        print(f'Epoch [{epoch}], ')
        # Log validation losses to MLflow
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)

        # Check for early stopping
        # pausls Set parameters for both early stopping methods sonst nimm die alte verison
        stop_threshold = 0.1           # Threshold for divergence
        max_diverge_count = 3          # Max number of epochs with diverging validation loss
        patience = 5                   # Number of epochs for validation loss plateau
        best_val_loss = float('inf')   # Initialize best validation loss
        diverge_count = 0              # Counter for divergence-based stopping
        patience_counter = 0           # Counter for plateau-based stopping     

        for epoch in range(num_epochs):
            # Training and validation steps
            # Assume epoch_loss_cel_train and epoch_loss_cel_val are calculated after each epoch        

            # Check divergence-based stopping
            if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
                diverge_count += 1
                if diverge_count >= max_diverge_count:
                    print(f"Early stopping at epoch {epoch+1} due to validation loss diverging.")
                    break
            else:
                diverge_count = 0  # Reset diverge count if validation loss is not diverging        

            # Check plateau-based stopping
            if epoch_loss_cel_val < best_val_loss:
                best_val_loss = epoch_loss_cel_val
                patience_counter = 0  # Reset plateau counter if validation loss improves
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1} due to lack of validation loss improvement.")
                    break
            # Print epoch results
            print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

    print("Training complete!")

    # Log the model
    mlflow.pytorch.log_model(model, "model")

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")



check if mlflow has got saved weights and model

In [34]:
import mlflow.pytorch

# Load the model from the specified run
loaded_model = mlflow.pytorch.load_model("runs:/0eff88ff672544bdb0221d0c6415595d/model")

# Print model parameters to verify
print(loaded_model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[ 1.3891e-02,  1.4984e-02, -1.5100e-02,  ..., -4.0584e-02,
           -4.2081e-02, -6.9670e-02],
          [ 4.6761e-03,  6.0934e-03,  1.4868e-02,  ...,  2.1132e-03,
           -1.9909e-02, -3.7592e-02],
          [ 2.2779e-02,  2.3694e-02,  1.6083e-02,  ...,  1.0281e-01,
            6.3552e-02,  5.3003e-02],
          ...,
          [-4.3460e-04,  2.7882e-02, -1.0256e-02,  ..., -1.2746e-01,
           -7.5680e-02,  8.9530e-03],
          [ 4.1879e-03,  4.8122e-02,  6.2086e-02,  ...,  2.4300e-02,
           -3.2765e-02, -1.4657e-02],
          [-7.9303e-02, -3.2265e-02, -1.7845e-02,  ...,  3.5516e-02,
            2.3292e-02,  2.5605e-03]],

         [[-1.7885e-02,  1.1738e-02,  2.4101e-02,  ...,  5.4043e-02,
            4.4959e-02, -8.3943e-03],
          [-7.1654e-03,  1.9135e-02,  6.7894e-02,  ...,  1.5946e-01,
            1.4703e-01,  1.2088e-01],
          [-4.5564e-02, -7.5978e-02, -8.9691e-02,  ...,  1.2107e-01,
            1.6792e-01,  1.7